# Make figures for `pymathutils.mesh` submodule docs

In [2]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
ply_dir = (proj_dir / "data" / "example_ply").resolve()
sys.path.insert(0, str(proj_dir))



hex_patch_he_ply = f"{ply_dir}/hex_patch_he.ply"
hex_patch_vf_ply = f"{ply_dir}/hex_patch_vf.ply"
annulus_he_ply = f"{ply_dir}/annulus_he.ply"
annulus_vf_ply = f"{ply_dir}/annulus_vf.ply"
plane_6_he_ply = f"{ply_dir}/nice_plane/plane_0000006_he.ply"
plane_24_ply = f"{ply_dir}/nice_plane/plane_0000024_he.ply"
dumbbell_coarse_he_ply = f"{ply_dir}/dumbbell_coarse_he.ply"

## Simple plot

In [5]:
import sys
from pathlib import Path
proj_dir = str(Path("..").resolve())
sys.path.insert(0, proj_dir)

from pymathutils.mesh.pyutils import HalfEdgeMesh
from src.python.mesh_viewer import MeshViewer

ply_path = f"{proj_dir}/data/example_ply/nice_plane/plane_0000006_he.ply"
viewer_kwargs = {
    "image_dir": "output/half_edge_hexagon",
    "image_index_length": 6,
    "show_wireframe_surface": False,
    "show_face_colored_surface": True,
    "show_vertices": False,
    "show_half_edges": True,
    "rgba_face": (1.0, 0.7431, 0.0, 1.0),
    "rgba_edge": (0.0, 0.0, 0.0, 1.0),
    "rgba_half_edge": (0.5804, 0.0, 0.8275, 1.0),
    "rgb_background": (1, 1, 1),
    "view": {
        "azimuth": 10.0,
        "elevation": 90.0,
        "distance": 4.0,
        "focalpoint": (0, 0, 0),
    },
}
m = HalfEdgeMesh.load_ply(ply_path)
mv = MeshViewer(m, **viewer_kwargs)
# plotter = mv.pv_options_plot(show=True,
#         save=False,
#         title="",
#         show_wireframe_surface=False,
#         show_face_colored_surface=True,
#         show_vertex_colored_surface=False,
#         show_vertices=False,
#         show_half_edges=True,
#         show_vector_fields=False,
#         show_plot_axes=False,
#         view=viewer_kwargs["view"],
#         figsize=(2180, 2180),
#         fig_path=viewer_kwargs["fig_path"],
#                             )

plotter = mv.pv_plot(show=True, save=False,)



In [5]:

def plot_hem():
    import pyvista as pv
    from src.python.mesh_viewer import get_half_edge_vector_field
    import numpy as np
    from pymathutils.mesh.pyutils import HalfEdgeMesh

    ply_path = f"{proj_dir}/data/example_ply/nice_plane/plane_0000006_he.ply"
    m = HalfEdgeMesh.load_ply(ply_path)
    numF = len(m.h_right_F)
    numH = len(m.v_origin_H)

    plotter = pv.Plotter(notebook=False)
    
    # Define PyVista surface mesh
    F3 = np.hstack([np.full((numF, 1), 3), m.V_cycle_F]).astype(int)
    pv_m = pv.PolyData(m.xyz_coord_V, faces=F3)
    # rgba_F = (self.rgba_F).round().astype(INT_TYPE)
    rgba_F = np.zeros((numF, 4), dtype=int)  # color faces of pv_m
    rgba_F[:, 0], rgba_F[:, 1], rgba_F[:, 2], rgba_F[:, 3] = 255, 189, 0, 255
    pv_m.cell_data["RGBA"] = rgba_F
    plotter.add_mesh(pv_m, scalars="RGBA", rgba=True, show_edges=True)
    # Define PyVista half-edge vector field
    pts_H, vecs_H = get_half_edge_vector_field(m)
    # rgba_H = (self.rgba_H).round().astype(INT_TYPE)
    rgba_H = np.zeros((numH, 4), dtype=int)
    rgba_H[:, 0], rgba_H[:, 1], rgba_H[:, 2], rgba_H[:, 3] = 148, 0, 211, 255
    vecs_H_source = pv.Arrow(
        tip_length=0.125,  # Arrow tip length (0.0-1.0)
        tip_radius=0.025,  # Arrow tip radius
        tip_resolution=5,  # Tip smoothness
        shaft_radius=0.00625,  # Shaft thickness
        shaft_resolution=3,  # Shaft smoothness
    )
    pv_pts_H = pv.PolyData(pts_H)
    pv_pts_H["half_edges"] = vecs_H
    pv_pts_H["RGBA"] = rgba_H
    pv_H = pv_pts_H.glyph(
        orient="half_edges",
        scale="half_edges",
        factor=1,
        geom=vecs_H_source,
    )
    plotter.add_mesh(pv_H, scalars="RGBA", rgba=True, show_edges=False)
    
    # plotter = pv.Plotter(notebook=True)
    # plotter.show(jupyter_backend='trame')
    plotter.show()
    return plotter



p = plot_hem()

## Next half-edge along face boundary

In [1]:
def next_cycle():
    import sys
    from pathlib import Path
    proj_dir = str(Path("..").resolve())
    sys.path.insert(0, proj_dir)
    
    from pymathutils.mesh.pyutils import HalfEdgeMesh
    from src.python.mesh_viewer import MeshViewer
    
    ply_path = f"{proj_dir}/data/example_ply/nice_plane/plane_0000006_he.ply"

    viewer_kwargs = {
        "image_index_length": 6,
        "image_dir": f"{proj_dir}/output/next_cycle",
        "show_wireframe_surface": True,
        "show_face_colored_surface": True,
        "show_vertices": False,
        "show_half_edges": True,
        "rgba_face": (1.0, 0.7431, 0.0, 1.0),
        "rgba_edge": (0.0, 0.0, 0.0, 1.0),
        "rgba_half_edge": (0.5804, 0.0, 0.8275, 0.1),
        "rgb_background": (1, 1, 1),
        "view": {
            "azimuth": 0.0,
            "elevation": 90.0,
            "distance": 4.0,
            "focalpoint": (0, 0, 0),
        },
    }
    m = HalfEdgeMesh.load_ply(ply_path)
    rgba = (0.5804, 0.0, 0.8275, 1.0)
    rgba_clear = (0.5804, 0.0, 0.8275, 0.1)
    mv = MeshViewer(m, **viewer_kwargs)
    # h0 = 0
    # h1 = m.h_next_h(h0)
    # h2 = m.h_next_h(h1)
    # mv = MeshViewer(m, **viewer_kwargs)
    # mv.update_rgba_H(rgba, [h0])
    # mv.plot()
    # mv.update_rgba_H(rgba_clear, [h0])
    # mv.update_rgba_H(rgba, [h1])
    # mv.plot()
    # mv.update_rgba_H(rgba_clear, [h1])
    # mv.update_rgba_H(rgba, [h2])
    # mv.plot()
    h = 0
    for _ in range(5):
        mv.update_rgba_H(rgba, [h])
        mv.pv_plot(save=True, show=False)
        mv.update_rgba_H(rgba_clear, [h])
        h = m.h_next_h(h)


next_cycle()

3.0